In [1]:
import pandas as pd
import numpy as np
import os
import scipy
import itertools
import seaborn as sns
import matplotlib.pyplot as plt    
import xgboost as xgb
iv=['np'    , 'os'     ,'pd'    , 'plt'     ,'tqdm'    , 'train'     ,'train_date'      , 'train_num' ,'data'    ]
def memchk(iv):
    import sys,operator
    ipython_vars=iv
    ,'warnings'    , 'xgboost']
    print sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)
    return sorted([{i:sys.getsizeof(globals().get(i))/1000000.0} for i in ipython_vars])
%matplotlib inline
from sklearn.metrics import confusion_matrix
import warnings
%load_ext autotime
warnings.filterwarnings('ignore')
from tqdm import tqdm
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
plt.style.use('classic')
from contigency import Ttable,table, plot 

C:\Users\sushant\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
def rmsle(predicted,real):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

time: 150 ms


In [3]:
data = pd.read_csv(r'C:\Users\sushant\Documents\Datasets\NYC\data_clean.csv').drop('Unnamed: 0',1)

time: 26.2 s


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083778 entries, 0 to 2083777
Data columns (total 22 columns):
dropoff_datetime         object
dropoff_latitude         float64
dropoff_longitude        float64
id                       object
passenger_count          int64
pickup_datetime          object
pickup_latitude          float64
pickup_longitude         float64
store_and_fwd_flag       object
trip_duration            float64
vendor_id                int64
dropoff_datetimeyear     float64
dropoff_datetimemonth    float64
dropoff_datetimeday      float64
dropoff_datetimehour     float64
dropoff_datetimedate     object
pickup_datetimeyear      int64
pickup_datetimemonth     int64
pickup_datetimeday       int64
pickup_datetimehour      int64
pickup_datetimedate      object
Train                    int64
dtypes: float64(9), int64(7), object(6)
memory usage: 349.8+ MB
time: 150 ms


In [5]:
data[data['Train']==1][['trip_duration']].mean()

trip_duration    959.492273
dtype: float64

time: 533 ms


In [6]:
data[data['Train']==1].index

Int64Index([      0,       1,       2,       3,       4,       5,       6,
                  7,       8,       9,
            ...
            1458634, 1458635, 1458636, 1458637, 1458638, 1458639, 1458640,
            1458641, 1458642, 1458643],
           dtype='int64', length=1458644)

time: 412 ms


In [7]:
data[data['Train']==0].index

Int64Index([1458644, 1458645, 1458646, 1458647, 1458648, 1458649, 1458650,
            1458651, 1458652, 1458653,
            ...
            2083768, 2083769, 2083770, 2083771, 2083772, 2083773, 2083774,
            2083775, 2083776, 2083777],
           dtype='int64', length=625134)

time: 181 ms


In [8]:
round(len(data[data['Train']==1])*.2)
#data['Valid']=0

291729.0

time: 879 ms


In [9]:
data['Valid']=0

time: 258 ms


In [10]:
data['Valid'][data['Train']==0]=2

time: 4.56 s


In [11]:
print data['Train'].value_counts()
data['Valid'].value_counts()

1    1458644
0     625134
Name: Train, dtype: int64


0    1458644
2     625134
Name: Valid, dtype: int64

time: 269 ms


In [12]:
int(round(len(data[data['Train']==1])*.2))

291729

time: 398 ms


In [13]:
data['Valid'][(data['Train']==1)&(data.index.isin(range(int(round(len(data[data['Train']==1])*.2)))))] = 1

time: 1.06 s


In [14]:
print data['Train'].value_counts()
data['Valid'].value_counts()

1    1458644
0     625134
Name: Train, dtype: int64


0    1166915
2     625134
1     291729
Name: Valid, dtype: int64

time: 80 ms


In [15]:
data[data['Valid']==0][['trip_duration']].mean()

trip_duration    962.072552
dtype: float64

time: 522 ms


In [16]:
data[data['Valid']==1][['trip_duration']].mean()[0]

949.17116570515782

time: 271 ms


In [17]:
pred=data[data['Valid']==1][['trip_duration']]

time: 90 ms


In [18]:
pred['mean_pred']=data[data['Valid']==0][['trip_duration']].mean()[0]

time: 1.04 s


In [19]:
pred['err']=pred['mean_pred']-pred['trip_duration']

time: 691 ms


In [20]:
pred['sqr_err']=pred['err']*pred['err']

time: 69 ms


In [21]:
pred['err'].describe()

count    291729.000000
mean         12.901386
std        3124.816642
min      -85427.927448
25%        -111.927448
50%         300.072552
75%         566.072552
max         961.072552
Name: err, dtype: float64

time: 463 ms


In [ ]:
pred.plot(x='trip_duration',y='err',kind='scatter')

In [22]:
pred.describe()

,trip_duration,mean_pred,err,sqr_err
count,291729.000000,2.917290e+05,291729.000000,2.917290e+05
mean,949.171166,9.620726e+02,12.901386,9.764612e+06
std,3124.816642,7.958092e-13,3124.816642,2.557891e+08
min,1.000000,9.620726e+02,-85427.927448,5.263791e-03
25%,396.000000,9.620726e+02,-111.927448,5.715569e+04
50%,662.000000,9.620726e+02,300.072552,2.096975e+05
75%,1074.000000,9.620726e+02,566.072552,4.423215e+05
max,86390.000000,9.620726e+02,961.072552,7.297931e+09


time: 396 ms


In [23]:
pred[pred['trip_duration']<20000].describe()

,trip_duration,mean_pred,err,sqr_err
count,291329.000000,2.913290e+05,291329.000000,2.913290e+05
mean,836.984094,9.620726e+02,125.088458,4.540090e+05
std,662.090191,9.094963e-13,662.090191,1.616584e+06
min,1.000000,9.620726e+02,-17476.927448,5.263791e-03
25%,396.000000,9.620726e+02,-109.927448,5.715569e+04
50%,661.000000,9.620726e+02,301.072552,2.089153e+05
75%,1072.000000,9.620726e+02,566.072552,4.409924e+05
max,18439.000000,9.620726e+02,961.072552,3.054430e+08


time: 146 ms


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083778 entries, 0 to 2083777
Data columns (total 23 columns):
dropoff_datetime         object
dropoff_latitude         float64
dropoff_longitude        float64
id                       object
passenger_count          int64
pickup_datetime          object
pickup_latitude          float64
pickup_longitude         float64
store_and_fwd_flag       object
trip_duration            float64
vendor_id                int64
dropoff_datetimeyear     float64
dropoff_datetimemonth    float64
dropoff_datetimeday      float64
dropoff_datetimehour     float64
dropoff_datetimedate     object
pickup_datetimeyear      int64
pickup_datetimemonth     int64
pickup_datetimeday       int64
pickup_datetimehour      int64
pickup_datetimedate      object
Train                    int64
Valid                    int64
dtypes: float64(9), int64(8), object(6)
memory usage: 365.7+ MB
time: 8 ms


In [25]:
#data.to_csv(r'C:\Users\sushant\Documents\Datasets\NYC\data_clean.csv',index=False)

time: 1 ms


In [32]:
printmd('### Validation Err\n# %f'%float(rmsle(pred['mean_pred'],pred['trip_duration'])))

### Validation Err
# 0.891920

time: 9.85 s
